In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json

import numpy as np
import pandas as pd
import yaml
from torch.utils.data import Dataset, DataLoader
import torch
import glob
import pandas as pd
from img2vec import Img2Vec
from PIL import Image


class CorrnetDataset(Dataset):

    def __init__(self, img, txt):
        self.img = img
        self.txt = txt
        if self.img.shape[0] != self.txt.shape[0]:
            raise Exception("Different no. of samples")

    def __len__(self):
        return self.img.shape[0]

    def __getitem__(self, index):
        _img = self.img[index]
        _txt = self.txt[index]

        return _img, _txt


def read_dataset(file_path):
    dataset_images = pd.DataFrame()
    vector_images = []
    images_name = []
    img2vec = Img2Vec('resnet-18', 'default', 512, cuda=True)

    i = 0
    for image_name in glob.glob(file_path + "/images/*.jpg"):
        i += 1
        img = Image.open(image_name).convert('RGB')
        vector_images.append(img2vec.get_vec(img))
        images_name.append('images/' + image_name.split('\\')[-1])

        if i % 100 == 1:
            print(i)

    dataset_images['image'] = images_name
    dataset_images['img_vec'] = vector_images

    return dataset_images



In [ ]:
dataset_csv = pd.read_csv('/content/drive/MyDrive/final_nlp_data/dataset.csv')
dataset_csv.head()

,Unnamed: 0,image,caption
0,0,images/wYzLGvNM_1.jpg,بازی ps4
1,1,images/wYzatbRC_1.jpg,خریدار PS4ایکس باکس XBOX پلی استیشن خریدPS3در محل
2,2,images/wYzD2v19_1.jpg,jbl charg 3 جی بی ال
3,3,images/wYzLG7Cq_1.jpg,شیائومی Poco M3 Pro 5G با حافظهٔ ۱۲۸ گیگابایت
4,4,images/wYyPmZKg_1.jpg,کامپیوتر کامل گیمینگ کیس مانیتور و کیبورد-PC


In [ ]:
import pickle
import glob

def compute_image_vec(image_name):
    img = Image.open(image_name).convert('RGB')
    return img2vec.get_vec(img)

dataset_images = pd.DataFrame()
vector_images = []
images_name = []
img2vec = Img2Vec('resnet-18', 'default', 512, cuda=False)

# images_list =  glob.glob('/content/drive/MyDrive/final_nlp_data/images/*')
# print(len(images_list))

dataset = pd.read_pickle('/content/drive/MyDrive/final_nlp_data/dataset_images')
processed_files = dataset['images_name']
print(len(processed_files))

remain_files = [x for x in images_list if x not in list(processed_files)]
print(len(remain_files))

remain_files.remove('/content/drive/MyDrive/final_nlp_data/images/thisdir.zip')
before_dataset = pd.read_pickle('/content/drive/MyDrive/final_nlp_data/dataset_images')

dataset_images['images_name'] = remain_files
dataset_images['img_vec'] = dataset_images['images_name'].apply(compute_image_vec)


before_dataset = pd.concat([dataset_images, before_dataset], ignore_index=True, sort=False)
before_dataset.to_pickle('/content/drive/MyDrive/final_nlp_data/dataset_images')

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


48219
1002


In [ ]:
import pandas as pd
before_dataset = pd.read_pickle('/content/drive/MyDrive/final_nlp_data/dataset_images')
before_dataset

,images_name,img_vec
0,/content/drive/MyDrive/final_nlp_data/images/w...,"[0.21020284, 1.3185495, 3.0565457, 0.41348276,..."
1,/content/drive/MyDrive/final_nlp_data/images/w...,"[0.92521465, 0.60474706, 2.6438816, 0.07804908..."
2,/content/drive/MyDrive/final_nlp_data/images/w...,"[0.2805308, 0.3629043, 0.043653883, 0.07562307..."
3,/content/drive/MyDrive/final_nlp_data/images/w...,"[0.6482546, 0.9707212, 2.1181266, 0.11031342, ..."
4,/content/drive/MyDrive/final_nlp_data/images/w...,"[1.0718853, 0.12620959, 0.28851634, 0.9411607,..."
...,...,...
49215,/content/drive/MyDrive/final_nlp_data/images/w...,"[1.0894359, 0.07223655, 0.83618355, 0.58940077..."
49216,/content/drive/MyDrive/final_nlp_data/images/w...,"[1.4432899, 0.33126977, 0.20663112, 0.42309123..."
49217,/content/drive/MyDrive/final_nlp_data/images/w...,"[0.36027947, 0.7125931, 0.7088299, 1.1372142, ..."
49218,/content/drive/MyDrive/final_nlp_data/images/w...,"[1.611069, 0.30400014, 0.6737227, 0.48536238, ..."
